In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os
import boto3

In [ ]:
data_folder = "../data"
test_file_path = os.path.join(data_folder, "dev.csv")
df = pd.read_csv(test_file_path)
df['instruction'].head()

In [ ]:
from botocore.exceptions import ClientError
import json
result_data = []

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-west-2")

model_id = 'openai.gpt-oss-20b-1:0'

for index, row in df.iterrows():
    message = row['instruction']

    prompt = f"You are a helpful assistant that generates code snippets based on user instructions." \
                "The method signature should be according to the given in the example part.\n" \
                "Use the following guidlines in generation: \n" \
                " - Provide the code in the response only\n" \
                " - Do not output any other texts or test cases results.\n" \
                " - Include the necessary import statements on the top even if they are usually not needed like - re, typing, itertools, Split etc. \n" \
                " - Do not include any comments in the code\n" \
                " - Do not forget to consider the edge cases\n" \
                " - Avoid recursion if possible\n" \
                " - Lastly, verify the code has no compilations error and contains the necessary imports.\n"
    
    native_request = {
        "model": model_id, # You can omit this field
        "messages": [
            {
            "role": "system",
            "content": prompt
            },
            {
            "role": "user",
            "content": message
            }
        ],
        "max_completion_tokens": 1024,
        "temperature": 0.5
    }

    try:
        # Invoke the model with the request.
        response = client.invoke_model(modelId=model_id, body=json.dumps(native_request))

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

    # Decode the response body.
    model_response = json.loads(response["body"].read().decode('utf-8'))
    code_response = model_response['choices'][0]['message']['content'].split('</reasoning>')[1]

    result_data.append({
        "id": row['id'],
        "response": code_response
    })

    print("Completed:", row['id'])

result_df = pd.DataFrame(result_data)

In [ ]:
# Create a json from the result_df
result_json = result_df.to_json(orient="records", indent=4)
# save json in the data_folder os.path.join(data_folder, "submission.json")
with open("gpt_oss_20B_submission.json", "w") as json_file:
    json_file.write(result_json)